In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-text-classification1/data/train_5500.label.txt"
testDataPath = "s3://corpus-text-classification1/data/TREC_10.label.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    train_data = []
    with open(filePath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f.readlines():
            word = line.strip().split()
            label = word[0].split(":")[0]
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, cat_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [4]:
categories = ['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']
num_classes = len(categories)

cat_to_id = {'ABBR': 0, 'DESC': 1, 'ENTY': 2, 'HUM': 3, 'LOC': 4, 'NUM': 5}

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)

testData = readfile(testDataPath)
trainData = readfile(trainDataPath)

print(len(testData),len(trainData))
trainData = np.r_[trainData,testData]
np.random.shuffle(trainData)
len(trainData)

seq_length = 37

Loading GloVe!
Completed!
400000 100 400000
500 5452


In [5]:
def train_10_fold(train_data, categories):
    
    tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, cat_to_id, num_classes, seq_length)
    print(len(tx),len(tx[0]),len(tx[1]))
    
    fold_id = 0
    test_acc = []
    
    kf = KFold(n_splits=10)
    for train_i, test_i in kf.split(tx):
        fold_id += 1
        print("Fold: ", fold_id)
        test_acc.append(classifier.train(
            X_train=tx[train_i],
            y_train=ty[train_i],
            X_eval=tx[test_i],
            y_eval=ty[test_i],
            categories=categories,
            epochs=80
        ))
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [6]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.005)  # learning_rate = 0.001
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=5):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        

        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [7]:
class ZhouBLSTMCNNModel:
    '''
    Implementation proposal of: https://arxiv.org/pdf/1611.06639v1.pdf
    '''
    def __init__(self,
        embedding,
        em_drop_rate = 0.5,
        lstm_units   = 6,  # 300
        lstm_drop_rate = 0.2,
        conv_size    = (2, 2), # (3, 3)
        conv_filters = 3,  # 100
        pool_size    = (2, 2),
        pool_drop_rate = 0.4):
        '''Constructor.
        # Parameters:
        embedding: Numpy array representing the embedding.
        em_drop_rate: Drop rate after the embedding layer.
        lstm_units: Size of the internal states of the LSTM cells.
        lstm_drop_rate: Drop rate after the lstm layer.
        conv_size: Size of the convolutions.
        conv_filters: Number of convolutions filters.
        pool_size: Size for the max pooling layer.
        pool_drop_rate: Drop rate of the max pooling layer.
        '''
        self._embedding      = embedding
        self._em_drop_rate   = em_drop_rate
        self._lstm_units     = lstm_units
        self._lstm_drop_rate = lstm_drop_rate
        self._conv_size      = conv_size
        self._conv_filters   = conv_filters
        self._pool_size      = pool_size
        self._pool_drop_rate = pool_drop_rate

    def __call__(self, input):
        self._embedding_tf = self._create_embedding_layer(
            self._em_drop_rate, self._embedding, input)

        self._sequences_tf = self._create_blstm_layer(
            self._lstm_units,
            self._lstm_drop_rate,
            self._embedding_tf)

        self._convolution_tf = self._create_convolutional_layer(
            self._conv_size,
            self._conv_filters,
            self._sequences_tf)
        self._pooling_tf = self._create_maxpooling_layer(
            self._pool_size,
            self._pool_drop_rate,
            self._convolution_tf)

        self._flatten_tf = self._create_flatten_layer(self._pooling_tf)

        return self._flatten_tf

    def summary(self):
        print("embedding: " + str(self._embedding_tf.shape))
        print("lstm: " + str(self._sequences_tf.shape))
        print("conv: " + str(self._convolution_tf.shape))
        print("pooling: " + str(self._pooling_tf.shape))
        print("flatten: " + str(self._flatten_tf.shape))

    def _create_embedding_layer(self, em_drop_rate, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return tf.nn.dropout(embedded_chars, em_drop_rate)

    def _create_blstm_layer(self, lstm_units, lstm_drop_rate, embedding):
        lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_units)
        sequence = tf.unstack(embedding, axis=1)
        hs, _, _ = tf.nn.static_bidirectional_rnn(lstm_cell, lstm_cell,
            sequence,
            dtype=tf.float32)
        
        hs = tf.stack(
            values=hs,
            axis=1)
        ss = tf.reduce_sum(
            tf.reshape(hs, shape=[-1, hs.shape[1], 2, lstm_units]),
            axis=2
        )

        return tf.nn.dropout(ss, lstm_drop_rate)

    def _create_convolutional_layer(self,
        conv_size, num_filters, tensor):
        
        print(str(tensor.shape))

        filter_heigth = conv_size[0]
        filter_width  = conv_size[1]

        filter_shape = [filter_heigth, filter_width,
            1, num_filters]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[num_filters]))

        tensor_expanded = tf.expand_dims(tensor, -1)
        conv = tf.nn.conv2d(
            input=tensor_expanded,
            filter=W,
            strides=[1,1,1,1],
            padding='VALID')

        bias = tf.nn.bias_add(conv, b)
        c = tf.nn.relu(bias)

        return c

    def _create_maxpooling_layer(self, size, pool_drop_rate, conv):
        pooled = tf.nn.max_pool3d(
            input=tf.expand_dims(conv, -1),
            ksize=[1, size[0], size[1], conv.shape[3], 1],
            strides=[1, size[0], size[1], conv.shape[3], 1],
            padding='VALID')
        
        return tf.nn.dropout(pooled, pool_drop_rate)

    def _create_flatten_layer(self, tensor):
        return tf.reshape(tensor, [-1, tensor.shape[1] * tensor.shape[2]])

In [8]:
word_vector = embedding.astype('float32')
model = ZhouBLSTMCNNModel(embedding=word_vector)

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

(?, 37, 6)
input: (?, 37)
embedding: (?, 37, 100)
lstm: (?, 37, 6)
conv: (?, 36, 5, 3)
pooling: (?, 18, 2, 1, 1)
flatten: (?, 36)
output: (?, 6)


In [ ]:
train_10_fold(trainData, categories)

5952 37 37
Fold:  1
Epoch: 1, Train Loss:  1.724, Train Acc: 21.33%, Val Loss:   1.67, Val Acc: 21.98%, Time: 24.10s *
Epoch: 2, Train Loss:  1.651, Train Acc: 24.06%, Val Loss:  1.553, Val Acc: 28.36%, Time: 20.53s *
Epoch: 3, Train Loss:  1.483, Train Acc: 34.49%, Val Loss:  1.396, Val Acc: 40.94%, Time: 20.59s *
Epoch: 4, Train Loss:  1.357, Train Acc: 41.82%, Val Loss:  1.351, Val Acc: 43.79%, Time: 20.70s *
Epoch: 5, Train Loss:   1.25, Train Acc: 48.20%, Val Loss:  1.295, Val Acc: 47.32%, Time: 20.78s *
Epoch: 6, Train Loss:  1.164, Train Acc: 52.47%, Val Loss:  1.275, Val Acc: 51.51%, Time: 20.53s *
Epoch: 7, Train Loss:  1.061, Train Acc: 57.42%, Val Loss:   1.26, Val Acc: 53.19%, Time: 20.40s *
Epoch: 8, Train Loss: 0.9988, Train Acc: 60.11%, Val Loss:  1.296, Val Acc: 53.52%, Time: 20.41s *
Epoch: 9, Train Loss:  0.968, Train Acc: 61.93%, Val Loss:  1.226, Val Acc: 57.38%, Time: 20.69s *
Epoch: 10, Train Loss: 0.9302, Train Acc: 63.31%, Val Loss:  1.286, Val Acc: 56.21%, Time

/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch: 1, Train Loss:  1.746, Train Acc: 20.65%, Val Loss:  1.699, Val Acc: 20.13%, Time: 23.14s *
Epoch: 2, Train Loss:  1.631, Train Acc: 25.40%, Val Loss:  1.527, Val Acc: 35.07%, Time: 20.64s *
Epoch: 3, Train Loss:  1.372, Train Acc: 42.00%, Val Loss:  1.376, Val Acc: 45.13%, Time: 20.81s *
Epoch: 4, Train Loss:  1.227, Train Acc: 49.13%, Val Loss:   1.29, Val Acc: 45.47%, Time: 20.53s *
Epoch: 5, Train Loss:  1.135, Train Acc: 53.68%, Val Loss:  1.229, Val Acc: 50.50%, Time: 20.59s *
Epoch: 6, Train Loss:  1.072, Train Acc: 57.68%, Val Loss:  1.249, Val Acc: 54.19%, Time: 20.43s *
Epoch: 7, Train Loss:  1.009, Train Acc: 61.25%, Val Loss:  1.243, Val Acc: 53.69%, Time: 20.53s 
Epoch: 8, Train Loss: 0.9535, Train Acc: 63.64%, Val Loss:  1.282, Val Acc: 56.71%, Time: 20.47s *
Epoch: 9, Train Loss: 0.9173, Train Acc: 65.12%, Val Loss:  1.229, Val Acc: 56.38%, Time: 20.62s 
Epoch: 10, Train Loss: 0.8569, Train Acc: 67.64%, Val Loss:  1.268, Val Acc: 58.72%, Time: 20.56s *
Epoch: 11, 

Epoch: 1, Train Loss:  1.712, Train Acc: 21.31%, Val Loss:  1.681, Val Acc: 22.86%, Time: 24.36s *
Epoch: 2, Train Loss:   1.67, Train Acc: 21.90%, Val Loss:  1.663, Val Acc: 23.53%, Time: 20.61s *
Epoch: 3, Train Loss:  1.615, Train Acc: 27.18%, Val Loss:  1.576, Val Acc: 32.61%, Time: 20.77s *
Epoch: 4, Train Loss:  1.445, Train Acc: 38.39%, Val Loss:  1.462, Val Acc: 38.15%, Time: 20.67s *
Epoch: 5, Train Loss:  1.298, Train Acc: 44.69%, Val Loss:  1.334, Val Acc: 41.51%, Time: 20.78s *
Epoch: 6, Train Loss:  1.166, Train Acc: 54.98%, Val Loss:  1.335, Val Acc: 48.91%, Time: 20.65s *
Epoch: 7, Train Loss:  1.081, Train Acc: 59.10%, Val Loss:  1.309, Val Acc: 53.61%, Time: 20.87s *
Epoch: 8, Train Loss: 0.9883, Train Acc: 62.86%, Val Loss:  1.214, Val Acc: 56.47%, Time: 20.61s *
Epoch: 9, Train Loss: 0.9501, Train Acc: 64.68%, Val Loss:  1.407, Val Acc: 57.65%, Time: 20.56s *
Epoch: 10, Train Loss: 0.9111, Train Acc: 67.19%, Val Loss:  1.472, Val Acc: 56.64%, Time: 20.50s 
Epoch: 11,

Epoch: 37, Train Loss: 0.5334, Train Acc: 84.67%, Val Loss:  2.107, Val Acc: 65.04%, Time: 20.68s 
Epoch: 38, Train Loss: 0.5145, Train Acc: 84.69%, Val Loss:  2.019, Val Acc: 67.56%, Time: 20.84s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       0.00      0.00      0.00         7
        DESC       0.52      0.68      0.59       127
        ENTY       0.70      0.56      0.62       151
         HUM       0.72      0.64      0.68       118
         LOC       0.59      0.66      0.63       100
         NUM       0.83      0.83      0.83        92

   micro avg       0.65      0.65      0.65       595
   macro avg       0.56      0.56      0.56       595
weighted avg       0.66      0.65      0.65       595

Confusion Matrix...
[[ 0  7  0  0  0  0]
 [ 0 86 12  5 19  5]
 [ 0 28 85 22 12  4]
 [ 0 19 15 76  6  2]
 [ 0 21  7  1 66  5]
 [ 0  5  2  1  8 76]]
Fold:  6
Epoch: 1, Train Los

Epoch: 40, Train Loss: 0.5595, Train Acc: 82.84%, Val Loss:  1.641, Val Acc: 60.67%, Time: 27.35s 
Epoch: 41, Train Loss: 0.5582, Train Acc: 83.25%, Val Loss:  1.559, Val Acc: 64.20%, Time: 27.27s 
Epoch: 42, Train Loss: 0.5558, Train Acc: 83.53%, Val Loss:  1.484, Val Acc: 64.87%, Time: 27.48s *
Epoch: 43, Train Loss: 0.5359, Train Acc: 84.01%, Val Loss:  1.584, Val Acc: 62.86%, Time: 27.47s 
Epoch: 44, Train Loss: 0.5306, Train Acc: 84.29%, Val Loss:  1.686, Val Acc: 59.33%, Time: 27.46s 
Epoch: 45, Train Loss: 0.5062, Train Acc: 84.85%, Val Loss:  1.646, Val Acc: 62.18%, Time: 27.64s 
Epoch: 46, Train Loss: 0.5024, Train Acc: 84.87%, Val Loss:  1.604, Val Acc: 64.87%, Time: 27.31s 
Epoch: 47, Train Loss: 0.5245, Train Acc: 84.78%, Val Loss:  1.524, Val Acc: 65.04%, Time: 27.93s *
Epoch: 48, Train Loss: 0.5175, Train Acc: 84.65%, Val Loss:  1.804, Val Acc: 63.87%, Time: 27.62s 
Epoch: 49, Train Loss: 0.5182, Train Acc: 85.09%, Val Loss:  1.597, Val Acc: 66.22%, Time: 27.84s *
Epoch: 

Epoch: 1, Train Loss:  1.693, Train Acc: 20.82%, Val Loss:  1.652, Val Acc: 22.35%, Time: 30.96s *
Epoch: 2, Train Loss:  1.667, Train Acc: 21.70%, Val Loss:  1.628, Val Acc: 25.04%, Time: 27.24s *
Epoch: 3, Train Loss:  1.606, Train Acc: 28.83%, Val Loss:  1.521, Val Acc: 35.29%, Time: 27.71s *
Epoch: 4, Train Loss:  1.458, Train Acc: 36.75%, Val Loss:  1.425, Val Acc: 39.33%, Time: 27.77s *
Epoch: 5, Train Loss:  1.289, Train Acc: 46.19%, Val Loss:  1.351, Val Acc: 44.20%, Time: 27.75s *
Epoch: 6, Train Loss:  1.149, Train Acc: 54.20%, Val Loss:  1.273, Val Acc: 46.39%, Time: 28.06s *
Epoch: 7, Train Loss:  1.061, Train Acc: 57.74%, Val Loss:  1.263, Val Acc: 50.76%, Time: 27.45s *
Epoch: 8, Train Loss: 0.9961, Train Acc: 61.14%, Val Loss:  1.277, Val Acc: 53.45%, Time: 27.87s *
Epoch: 9, Train Loss: 0.9458, Train Acc: 63.12%, Val Loss:  1.295, Val Acc: 54.29%, Time: 27.27s *
Epoch: 10, Train Loss: 0.9106, Train Acc: 65.40%, Val Loss:  1.228, Val Acc: 55.46%, Time: 27.39s *
Epoch: 11